In [1]:
!pip install /kaggle/input/icecube-scripts/icecube-scripts/polars-0.16.13-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl -q
#!pip install datasets --no-index --find-links=file:///kaggle/input/hf-ds -U
!pip install ../input/icecube-scripts/icecube-scripts/entmax-1.1-py3-none-any.whl -q
!pip install ../input/icecube-scripts/icecube-scripts/einops-0.6.0-py3-none-any.whl -q
!pip install ../input/icecube-scripts/icecube-scripts/x_transformers-1.8.2-py3-none-any.whl -q
import subprocess
from pathlib import Path


# Install packages
whls = [
    "/kaggle/input/pytorchgeometric/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl",
    "/kaggle/input/pytorchgeometric/torch_scatter-2.1.0-cp37-cp37m-linux_x86_64.whl",
    "/kaggle/input/pytorchgeometric/torch_sparse-0.6.16-cp37-cp37m-linux_x86_64.whl",
    "/kaggle/input/pytorchgeometric/torch_spline_conv-1.2.1-cp37-cp37m-linux_x86_64.whl",
    "/kaggle/input/pytorchgeometric/torch_geometric-2.2.0-py3-none-any.whl",
    "/kaggle/input/pytorchgeometric/ruamel.yaml-0.17.21-py3-none-any.whl",
]

for w in whls:
    print("Installing", w)
    subprocess.call(["pip", "install", w, "--no-deps", "--upgrade", "-q"])

import sys
sys.path.append("/kaggle/input/graphnet/graphnet-main/src")
sys.path.append("/kaggle/input/")
sys.path.append('/kaggle/input/icecube-scripts/icecube-scripts/')
#sys.path.append('/kaggle/input/icecubescripts/')

Installing /kaggle/input/pytorchgeometric/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl


Installing /kaggle/input/pytorchgeometric/torch_scatter-2.1.0-cp37-cp37m-linux_x86_64.whl


Installing /kaggle/input/pytorchgeometric/torch_sparse-0.6.16-cp37-cp37m-linux_x86_64.whl


Installing /kaggle/input/pytorchgeometric/torch_spline_conv-1.2.1-cp37-cp37m-linux_x86_64.whl


Installing /kaggle/input/pytorchgeometric/torch_geometric-2.2.0-py3-none-any.whl


Installing /kaggle/input/pytorchgeometric/ruamel.yaml-0.17.21-py3-none-any.whl


In [2]:
import gc, os, random, math
import numpy as np
from tqdm.notebook import tqdm
from functools import partial

from data_test import *
#from encoders import EncoderWithDirectionReconstructionV22,EncoderWithDirectionReconstructionV23
from baselineV3_SE_globalrel_d32_2 import DeepIceModel as TransformerV3_2

In [3]:
MODELS = [
    ['/kaggle/input/icecube-weights/icecube-weights/B_MODEL_32.pth',
     partial(TransformerV3_2, dim=768, dim_base=192, depth=12, head_size=32), 0.20],#0.08254897],
    ['/kaggle/input/icecube-weights/icecube-weights/B_MODEL_B_64.pth',  
     partial(TransformerV3_2, dim=768, dim_base=192, depth=12, head_size=64), 0.60],#0.15350807],
    ['/kaggle/input/icecube-weights/icecube-weights/B_MODEL_B_64_REL.pth',  
     partial(TransformerV3_2, dim=768, dim_base=192, depth=12, head_size=32, n_rel=4), 0.60],#0.19367443],
    #['/kaggle/input/exp-24-ft-md/V22FT4_7.pth',
    # partial(EncoderWithDirectionReconstructionV22, dim=384, dim_base=128, depth=8, head_size=32),
    # 0.03735429],
#     ['/kaggle/input/exp-24-ft-md/V22FT6_1.pth',
#      partial(EncoderWithDirectionReconstructionV22, dim=384, dim_base=128, depth=8, head_size=32),
#      0.23597202],
#     ['/kaggle/input/exp-24-ft-md/V23FT5_6.pth',
#      partial(EncoderWithDirectionReconstructionV23, dim=768, dim_base=192, depth=12, head_size=64),
#      0.3342965],
]

PATH = '/kaggle/input/icecube-neutrinos-in-deep-ice/'
ICE_PROPERTIES = '/kaggle/input/icecube-scripts/icecube-scripts/'

NUM_WORKERS = 2
bs = 16 #64
L = 768 #512
SEED = 2023

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
ds = IceCubeDataset(PATH,ICE_PROPERTIES,L=L)
len_sampler = LenMatchBatchSampler(torch.utils.data.SequentialSampler(ds),batch_size=bs, drop_last=False)
dl = DataLoader(ds, batch_sampler=len_sampler, num_workers=0)

models,weights = [],[]
for path,Model,w in MODELS:
    print('loading:',path)
    model = Model()
    model.load_state_dict(torch.load(path,map_location=torch.device('cpu')))
    model.eval()
    model.to(device)
    models.append(model)
    weights.append(w)
weights = torch.FloatTensor(weights)
weights /= weights.sum()

loading: /kaggle/input/icecube-weights/icecube-weights/B_MODEL_32.pth
loading: /kaggle/input/icecube-weights/icecube-weights/B_MODEL_B_64.pth
loading: /kaggle/input/icecube-weights/icecube-weights/B_MODEL_B_64_REL.pth


In [5]:
preds = []
for x in tqdm(dl):
    with torch.no_grad():
        with torch.cuda.amp.autocast(enabled=True):
            x = dict_to(x,device)
            # torch.nan_to_num(model(x)).clip(-1000, 1000).shape => torch.Size([3, 3])
            p = (torch.stack([torch.nan_to_num(model(x)).clip(-1000,1000) 
                              for model in models],-1).cpu()*weights).sum(-1)
    p = get_val(p).numpy()
    for pi,idx in zip(p,x['idx']):
        preds.append({'event_id':idx.cpu().item(), 'azimuth':pi[0], 'zenith':pi[1]})
        
df = pd.read_parquet(os.path.join(PATH,'sample_submission.parquet'))
df = pd.merge(df['event_id'], pd.DataFrame(preds), on='event_id', how='left').fillna(value=0)
df.to_csv('submission.csv',index=False)
df.head()

  0%|          | 0/1 [00:00<?, ?it/s]

/kaggle/input/icecube-scripts/icecube-scripts/data_test.py:24: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  L = max(1,L // 16)


,event_id,azimuth,zenith
0,2092,6.122199,1.330500
1,7344,3.431445,2.496783
2,9482,4.685464,1.543473
